firebase rpc wrapper?


In [ ]:
var cors = require('cors')({origin: true});
var process = require('process');
var functions = require('firebase-functions');
var admin = require('firebase-admin');
var importer = require('../Core');
var getResult = importer.import('rpc result');

try {
    admin.initializeApp(functions.config().firebase);
} catch (e) {
    console.log(e);
}

if(typeof module.exports === 'undefined') {
    module.exports = {};
}

module.exports.rpc = functions.https.onRequest((req, res) => {
    process.env.HOME = process.env.HOMEPATH = process.env.USERPROFILE = '';
    const config = functions.config();
    const services = Object.keys(config).reduce(k => {
        Object.keys(config[k]).forEach(s => {
            process.env[k.toUpperCase() + '_' + s.toUpperCase()] = config[k][s];
        });
    });
    
    cors(req, res, () => {
        return Promise.resolve([])
            .then(() => getResult({
                command: req.body['function'] || req.query['function'],
                result: importer.interpret(req.body['function'] || req.query['function']),
                body: req.method === 'POST' ? req.body : req.query,
                circles: ['Public']
            }))
            .then(r => {
                const result = Object.getOwnPropertyNames(r).reduce((alt, key) => {
                    alt[key] = r[key] + '';
                    return alt;
                }, {});
                console.error(result);
                return res.status(200).send(r);
            })
            // TODO: object assign error?
            .catch(e => { 
                const result = Object.getOwnPropertyNames(e).reduce((alt, key) => {
                    alt[key] = e[key] + '';
                    return alt;
                }, {});
                console.error(result);
                return res.status(500).send(result);
            });
    });
})


google cloud function rpc wrapper?


In [ ]:
var cookie = require('cookie');
var cors = require('cors')({origin: true});
var importer = require('../Core');
var getResult = importer.import('rpc result');
var getEnvironment = importer.import('get environment');

var expressToResult = (req, res) => getResult({
    command: (req.body || {})['function'] || (req.query || {})['function'],
    result: importer.interpret((req.body || {})['function'] || (req.query || {})['function']),
    body: req.method === 'POST' ? req.body : req.query,
    circles: ['Public']
})

var logResult = (r) => {
    // TODO: call storeResult()?
    var result = typeof r === 'object'
        ? Object.keys(r).reduce((alt, key) => {
            alt[key] = r[key] + '';
            return alt;
        }, {})
        : ('' + r);
    console.log(result);
    return result;
}

function expressHandler(req, res) {
    getEnvironment();
    req.cookies = cookie.parse(req.get('cookie') || '');
    res.setHeader('Cache-Control', 'private');
    cors(req, res, () => {});
    return Promise.resolve()
        .then(() => expressToResult(req, res))
        .then(r => (res.status(200), r))
        .catch(e => (res.status(500), e))
        .then(r => logResult(r))
        .then(r => res.send(r));
}

module.exports.handler = expressHandler;



test google express rpc handler?


In [ ]:
var importer = require('../Core');
var {handler} = importer.import('google cloud function rpc wrapper');

if(typeof $$ !== 'undefined') {
    $$.async();
    handler({
        headers: {origin: 'google.com'},
        get: r => null,
        query: {
            function: 'create a copy of study sauce template',
            email: 'megamindbrian@gmail.com'
        }
    }, {
        getHeader: h => {},
        setHeader: h => console.log(`header ${h}`),
        status: s => console.log(`status ${s}`),
        send: r => console.log(r)
    })
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


generic gcloud function handler?



In [9]:
var importer = require('../Core')
var {selectAst} = importer.import('select ast')
var {
    niceName, getExports, getParameters
} = importer.import(['nice name', 'get exports', 'get parameters'])

async function handler(req, res) {
    if(!req && process.stdout.isTTY) {
        res = {}
        req = {query: {}, body: {}, form: {}}
        Array.from(process.argv).forEach(arg => {
            req.query[arg.split('=')[0]] = arg.split('=').slice(1).join('=')
        })
    }
    res.set('Access-Control-Allow-Origin', '*')
    var parameters = Object.values(req.query || {})
        .concat(Object.values(req.body || {}))
        .concat(Object.values(req.form || {}))
    
    var func = require('./entry.js')
    return await func(...parameters)
        .then(r => !res ? console.log(r) : res.status(200).send(r))
        .catch(e => !res ? console.log(e) : res.status(500).send(e))
}

// use syntax to alter the function handler above
//  to pull out parameters and call the requested function
//  slightly simpler code than including the entire library
function makeHandler(entry) {
    var thisCell = importer.interpret('generic cloud function handler')
    if(!Array.isArray(entry)) {
        entry = [entry]
    }
    var exports = []
    var interpret = importer.interpret(entry)
    var handlers = interpret.map(cell => {
        // replace the require statement
        // TODO: make this part generic, moving vars to parameters
        var doc = selectAst('.', handler.toString())

        var exportsName = getExports(cell.code)[0]
        exports.push(exportsName + 'Handler')
        var functionStmt = selectAst(`
//FunctionDeclaration/Identifier[@name="handler"]`, doc)
        functionStmt.setAttribute('name', exportsName + 'Handler')
        var parameters = getParameters(cell.code).slice(1)

        // replace the parameters with names
        var replaceParams = selectAst('//VariableDeclaration', `
var parameters = [${parameters
            .map(p => `(req.body || {})['${p}'] || (req.form || {})['${p}'] || (req.query || {})['${p}']`)
            .join(',\n')}]`)
        
        var assignmentStmt = selectAst([`//VariableDeclaration[./*/*[@name = "parameters"]]`], doc)
        assignmentStmt[0].replaceWith(replaceParams)
        
        var requireStmt = selectAst(`//VariableDeclaration[./*/*[@name = "func"]]`, doc)
        var replaceRequire = selectAst('//VariableDeclaration', `
var ${exportsName} = require('./${niceName(cell)}')`)
        requireStmt.replaceWith(replaceRequire)
                
        // replace function call with new named
        var callStmt = selectAst(`//CallExpression/Identifier[@name = "func"]`, doc)
        callStmt.setAttribute('name', exportsName)
        return doc.ownerDocument.toString()
    }).join('\n\n')
    return `${handlers}

module.exports = {
    ${exports.join(',\n    ')}
}`
}

module.exports = {
    makeHandler,
    handler
}


importing nice name,get exports,get parameters,select ast - 4 cells - notebook.ipynb[13],syntax.ipynb[5],syntax.ipynb[7],syntax.ipynb[20]


{
  makeHandler: [Function: makeHandler],
  handler: [AsyncFunction: handler]
}

#### test google handler


In [21]:
var importer = require('../Core')
var {selectAst} = importer.import('select code tree')
var {handler, makeHandler} = importer.import('generic gcloud function handler')

//var testFunc = importer.interpret('load ckeditor')
//console.log(handler.toString())
//var doc = selectAst('.', handler.toString())
//console.log(assignmentStmt[0].ownerDocument.toString(assignmentStmt[0]))
makeHandler('load ckeditor')




importing generic gcloud function handler - 1 cell - google rpc.ipynb[3]
compiling /Users/briancullinan/jupyter_ops/Cloud Services/google rpc.ipynb[3] aka generic gcloud function handler with module,importer,exports,__dirname,__filename,$$
importing nice name,get exports,get parameters,select ast - 4 cells - notebook.ipynb[13],syntax.ipynb[5],syntax.ipynb[7],syntax.ipynb[20]


'async function gitEditorHandler(req, res) {\n' +
  "    res.set('Access-Control-Allow-Origin', '*');\n" +
  '    if (!req && process.stdout.isTTY) {\n' +
  '        req = {\n' +
  '            query: {},\n' +
  '            body: {},\n' +
  '            form: {}\n' +
  '        };\n' +
  '        Array.from(process.argv).forEach(arg => {\n' +
  "            req.query[arg.split('=')[0]] = arg.split('=').slice(1).join('=');\n" +
  '        });\n' +
  '    }\n' +
  '    var parameters = [\n' +
  "        req.body['url'] || req.form['url'] || req.query['url'],\n" +
  "        req.body['gist'] || req.form['gist'] || req.query['gist']\n" +
  '    ];\n' +
  "    var gitEditor = require('./load-ckeditor.js');\n" +
  '    return await gitEditor(...parameters).then(r => !res ? console.log(r) : res.status(200).send(r)).catch(e => !res ? console.log(e) : res.status(500).send(e));\n' +
  '}\n' +
  '\n' +
  'module.exports = {\n' +
  '    gitEditorHandler\n' +
  '}'